In [1]:
# TODO: append to requirements.txt
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer
import numpy as np
import pandas as pd
import sklearn

# Local Bin
import src.lib.config as config

In [2]:
print(config.DATASET)

/home/jeffmur/data/mdcd/parsed/gps-sanitized.csv


In [3]:
df = pd.read_csv(config.FM_MATRIX)
dfShape = df.shape
print(df.shape)

(2512822, 6)


In [4]:
inputs = keras.Input(shape=df.shape)


# TODO: identify timeSTEPS (delta for splitting by temporal resolution)

In [5]:
encoder = layer.LSTM(64, activation='tanh')(inputs)
encoder = layer.Dropout(0.2)(encoder)
encoder = layer.Dense(64, activation='relu')(encoder)
encoder = layer.Dropout(0.2)(encoder)
encoder = layer.Dense(200, activation='relu')(encoder)
encoder = layer.Dropout(0.2)(encoder)
encoder_out = layer.Dense(30, activation=None, name='encoder_out')(encoder)

In [12]:
from src.lstmae.ClusteringLayer import *

timesteps = int(dfShape[0] / 8)
n_features = dfShape[1]

In [13]:
clustering = ClusteringLayer(n_clusters=2, name='clustering', alpha=0.05)(encoder_out)
hidden = layer.RepeatVector(timesteps, name='Hidden')(encoder_out)
decoder = layer.Dense(200, activation='relu')(hidden)
decoder = layer.Dense(64, activation='relu')(decoder)
decoder = layer.LSTM(64, activation='tanh', return_sequences=True)(decoder)
output = layer.TimeDistributed(Dense(n_features), name='decoder_out')(decoder)

<bound method Layer.set_weights of <src.lstmae.ClusteringLayer.ClusteringLayer object at 0x7fbe800241d0>>


In [14]:
kmeans = KMeans(n_clusters=6, n_init=100) 
encoder_model = Model(inputs=inputs, outputs=encoder_out)
kmeans.fit(encoder_model.predict(dfShape))    
model = Model(inputs=inputs, outputs=[clustering, output])
clustering_model = Model(inputs=inputs, outputs=clustering)

ValueError: in user code:

    /home/jeffmur/dev/lstm/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/jeffmur/dev/lstm/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/jeffmur/dev/lstm/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/jeffmur/dev/lstm/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/jeffmur/dev/lstm/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/jeffmur/dev/lstm/venv/lib/pyt